In [1]:
######################
# 1. GEE Python API: #
######################
# GEE functions build up JSON dictionary of operations. Call .getInfo() to 
# evaluate the image/feature/collection object. Server returns object properties as json.
# Use in Jupyter Notebook for easy image previews.


import ee
ee.Initialize()

startdate = '2016-03-01'
enddate = '2016-05-30'

aoi = ee.Geometry.Polygon(
        [[[10.4644775390625, 51.17759467314004],
          [10.43975830078125, 50.83887470777369],
          [11.76910400390625, 50.840609151331336],
          [11.7279052734375, 51.17759467314004]]]);
print 'aoi area', aoi.area().getInfo(), 'sqm'

# Filter Sentinel 2 collection to aoi and dates.
S2 = (
    ee.ImageCollection('COPERNICUS/S2')
        .filterDate(startdate, enddate)
        .filterBounds(aoi)
)
print 'S2size:', S2.size().getInfo()

# Select Image of least cloudcover.
S2_0 = ee.Image(S2.sort('CLOUDY_PIXEL_PERCENTAGE', True).first())
bands = ['B4', 'B3', 'B2']
S2_0 = S2_0.select(bands).clip(aoi)

print 'CLOUDY_PIXEL_PERCENTAGE', S2_0.getInfo()['properties']['CLOUDY_PIXEL_PERCENTAGE']
product_id = S2_0.getInfo()['properties']['PRODUCT_ID']
print 'PRODUCT_ID', product_id

# Display preview image in Jupyter Notebook.
from IPython.display import Image, display, HTML
thumburl = S2_0.getThumbUrl({
                'min':0, 
                'max':3000          
            })
#print thumburl
img_thumb = Image(url=thumburl)
display(img_thumb)

# Export tif to gdrive.
task = ee.batch.Export.image(
    S2_0.select(bands), 
    'S2_0', 
    {'driveFolder':'test_a'})
task.start()

aoi area 3405812352.48 sqm
S2size: 37
CLOUDY_PIXEL_PERCENTAGE 0.1087
PRODUCT_ID S2A_OPER_PRD_MSIL1C_PDMC_20160502T191434_R065_V20160502T102027_20160502T102027


In [ ]:
##########################
# 2. GEE Javascript API: #
##########################
# Executable in browser interface only https://code.earthengine.google.com/
# Avoid .getInfo()! 


var startdate = '2016-03-01'
var enddate = '2016-05-30'

var aoi = ee.Geometry.Polygon(
        [[[10.4644775390625, 51.17759467314004],
          [10.43975830078125, 50.83887470777369],
          [11.76910400390625, 50.840609151331336],
          [11.7279052734375, 51.17759467314004]]]);
print('aoi area', aoi.area(), 'sqm')

// Filter Sentinel 2 collection to aoi and dates.
var S2 = ee.ImageCollection('COPERNICUS/S2')
          .filterDate(startdate, enddate)
          .filterBounds(aoi)
print('S2size:', S2.size())

// Select Image of least cloudcover.
var S2_0 = ee.Image(S2.sort('CLOUDY_PIXEL_PERCENTAGE', true).first())
var bands = ['B4', 'B3', 'B2']
S2_0 = S2_0.select(bands).clip(aoi)

print('CLOUDY_PIXEL_PERCENTAGE', S2_0.get('CLOUDY_PIXEL_PERCENTAGE'))
var product_id = S2_0.get('PRODUCT_ID')
print('PRODUCT_ID', product_id)

// Add preview Image to interactive map.
Map.addLayer(S2_0, {bands:bands, min:0, max:3000}, 'S2_0')
Map.centerObject(aoi)

// Export tif to gdrive.
Export.image.toDrive({image: S2_0.select(bands),
                      description: 'S2_0',
                      folder: 'test_a'})

<img  src="2017-03-16 02 32 05.png"/>